In [13]:
import pandas as pd
import numpy as np
import csv
import datetime
import itertools
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

from bravado.client import SwaggerClient
from bravado.exception import HTTPError

%load_ext autoreload

%autoreload 2
import helper
from Station import Station
from Query import Query

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Import client modules
These let us interface with the Swagger Client and pull info from EVE's API

In [2]:
# Lookup type and region info form the SDE
sde_client = SwaggerClient.from_url("http://evekit-sde.orbital.enterprises/latest/swagger.json",
                                   config={'ues_models': False, 'validate_responses' : False})

# We need a SwaggerClient for the market data colleciton service
mdc_client = SwaggerClient.from_url("https://evekit-market.orbital.enterprises//swagger",
                                   config={'use_models': False, 'validate_responses' : False})

### Specify stations and items of interest

In [3]:
# Choose items and stations of interest
# Station Names
names_station = ['Jita IV - Moon 4 - Caldari navy Assembly Plant', 
                 'Amarr VIII (Oris) - Emperor Family Academy',
                 'Rens VI - Moon 8 - Brutor Tribe Treasury']
#                  'Dodixie IX - Moon 20 - Federation Navy Assembly Plant']
#                  'Hek VIII - Moon 12 - Boundless Creation Factory',
#                  'Oursulaert III - Federation Navy Testing Facilities',
#                  'Tash-Murkon Prime II - Moon 1 - Kaalakiota Corporation Factory',
#                  'Agil VI - Moon 2 - CONCORD Logistic Support',
#                  'Tuomuta III - Ducia Foundry Mining Outpost']
# names_station_shorthand = ['Jita 4-4', 'Amarr',
#                            'Rens']
#                            'Dodixie']
#                            'Hek', 'Oursulaert',
#                            'Tash-Murkon', 'Agil', 'Tuomuta']
# dict_names = dict(zip(names_station, names_station_shorthand))
# Region names
names_region = ['The Forge', 'Domain']
# Item names
names_items = ["Biocells","Construction Blocks","Consumer Electronics"]

# Date range of interest (generally just today's date for simple trade find)
date = datetime.date.today() - datetime.timedelta(days = 1)

In [67]:
# Load items from csv
items_csv = "../data/rigs.csv"
names_items_full = list(np.array(pd.read_csv(items_csv).values.tolist()).flatten())
names_items_full[0:3]

['Anti-EM Pump', 'Anti-EM Screen Reinforcer', 'Anti-Explosive Pump']

In [71]:
names_items_full[0]

'Anti-EM Pump'

In [69]:
helper.get_type_id(sde_client, names_items_full[0])

IndexError: list index out of range

In [68]:
# Verify the accuracy of the rig typings
dict_items   = {x: helper.get_type_id(sde_client, x) for x in names_items_full}

IndexError: list index out of range

In [5]:
stations = {name: Station(sde_client, name) for name in names_station}

In [6]:
for key, station in stations.items():
    print(station)
    print(f"Station Name     : %s"%station.get_station_name())
    print(f"Station Shorthand: %s"%station.get_station_shorthand())
    print(f"Station ID       : %d"%station.get_station_id())
    print(f"Station Region ID: %s"%station.get_region_id())
    print()

Station Name     : Jita IV - Moon 4 - Caldari navy Assembly Plant
Station Shorthand: Jita
Station ID       : 60003760
Station Region ID: 10000002

Station Name     : Amarr VIII (Oris) - Emperor Family Academy
Station Shorthand: Amarr
Station ID       : 60008494
Station Region ID: 10000043

Station Name     : Rens VI - Moon 8 - Brutor Tribe Treasury
Station Shorthand: Rens
Station ID       : 60004588
Station Region ID: 10000030



In [7]:
stations.keys()

dict_keys(['Jita IV - Moon 4 - Caldari navy Assembly Plant', 'Amarr VIII (Oris) - Emperor Family Academy', 'Rens VI - Moon 8 - Brutor Tribe Treasury'])

In [8]:
# Look up the relative IDs for the names above
dict_station = {x: helper.get_station_id(sde_client, x) for x in names_station}
dict_region  = {x: helper.get_region_id(sde_client, x) for x in names_region}
dict_items   = {x: helper.get_type_id(sde_client, x) for x in names_items}

In [10]:
# Create a query with Jita as the master station
master_station = stations['Jita IV - Moon 4 - Caldari navy Assembly Plant']
query = Query(sde_client, mdc_client, master_station, stations, dict_items, date)

In [11]:
print(query.get_master_station().get_station_name())

Jita IV - Moon 4 - Caldari navy Assembly Plant


In [12]:
query.get_master_df().head()

,Items,Jita: buys,Jita: sells,Amarr: buys,Amarr: sells,Rens: buys,Rens: sells,Jita: Internal Spread,"Jita: Buy, Amarr: Sell","Amarr: Buy, Jita: Sell","Jita: Buy, Rens: Sell","Rens: Buy, Jita: Sell"
0,Biocells,10200.01,10820.90,8201.25,9999.24,300.00,8800.00,"[620.8899999999994, 5.737877625705805]","[-200.77000000000044, -2.007852596797361]","[2619.6499999999996, 24.209169292757533]","[-1400.0100000000002, -15.909204545454546]","[10520.9, 97.22758735410179]"
1,Construction Blocks,12000.00,12799.90,9901.53,12347.94,9505.00,12199.89,"[799.8999999999996, 6.249267572402907]","[347.9400000000005, 2.817797948483719]","[2898.369999999999, 22.643692528847875]","[199.88999999999942, 1.6384573959273354]","[3294.8999999999996, 25.741607356307465]"
2,Consumer Electronics,13101.87,13982.89,10959.64,13658.61,11601.17,NaN,"[881.0199999999986, 6.3007003559349934]","[556.7399999999998, 4.076110233764635]","[3023.25, 21.621066889605796]","[nan, nan]","[2381.7199999999993, 17.033102598962014]"


In [ ]:
mdc_client.MarketData.book(typeID=dict_items['Biocells'], regionID=dict_region['The Forge'],
                          date=str(date)).result()

In [ ]:
# Old query using ALL stations instead of a maste rstation
# query = Query(sde_client, mdc_client, dict_items, stations, date)

In [ ]:
query.get_master_df().head()

### Test Realm

In [ ]:
# Try to find a station's Region name with only the station info or region ID
sde_client.Map()

In [ ]:
# Test station comparisons function
test = ['a', 'b', 'c']
test_list = [test, test]
print(test_list)
print()

# test_cross = itertools.combinations(test_list, r=2)
# for item in test_cross:
#     print(item)

test_cross = [item for item in itertools.combinations(query.stations.keys(), r=2)]
test_cross

In [ ]:
# Test cross for external buy/sell spreads
test_df = query.get_master_df()
display(test_df.head())

In [ ]:
def calculate_spread(buy, sell):
        profit = sell-buy
        percentage = 100*(profit/sell)
        diff = [profit, percentage]
        
        return diff

In [ ]:
for stations in test_cross:
#     display(test_df.head())
    stations_dict = query.get_stations()
    np.nan
    station_a = stations_dict[stations[0]]
    station_b = stations_dict[stations[1]]
#     print(station_a.get_station_shorthand())
#     print(station_b.get_station_shorthand())
    
#     test_df.apply(lambda x: print(x[f'%s: sells'%station_b.get_station_shorthand()]), axis=1)

#     print(calculate_spread())

    test_df[f'%s: Buy, %s: Sell'%(station_a.get_station_shorthand(), station_b.get_station_shorthand())]  \
        = test_df.apply(lambda x: calculate_spread(x[f'%s: buys'%station_a.get_station_shorthand()],
                                                   x[f'%s: sells'%station_b.get_station_shorthand()]), axis=1)
        
    test_df[f'%s: Buy, %s: Sell'%(station_b.get_station_shorthand(), station_a.get_station_shorthand())]  \
        = test_df.apply(lambda x: calculate_spread(x[f'%s: buys'%station_b.get_station_shorthand()],
                                                   x[f'%s: sells'%station_a.get_station_shorthand()]), axis=1)
    
#     test_df[f'%s: Sell, %s: Buy'%(
#         station_a.get_station_shorthand(), station_b.get_station_shorthand())] = 5

In [ ]:
test_df.head()

In [ ]:
buy = 9601
sell = 10498
profit = sell-buy
print(profit)
percentage = 100*(profit/sell)
print(percentage)